In [1]:
%load_ext autoreload

%autoreload 2


In [2]:
import numpy
import pandas as pd
import seaborn as sns
import random
import math
import os

from IPython import display

pd.set_option('display.max_columns', None)

### Import model objects from their unique directories - easier to organize

from commuter_model.commuter_model import CommuterModel
from electrical_model.electrical_model import ElectricModel

In [3]:
# load ipums data to local variable (ipums_df_19)
ipums_df_19 = pd.read_csv("commuter_model/disaggregated_cleaned_ipums_data.csv",index_col=0)
ipums_df_19 = ipums_df_19[ipums_df_19['YEAR']==2019].reset_index(drop=True).copy()



### Iterations 
|WFH Status|Commuter Mode|Save Directory Name|Commuter Model Object|
|----|-----|-----|-----|
|None|Micro Heavy|NoWFH_Micro|commuter_model_nowfh_micro
|None|Transit Heavy|NoWFH_Transit|commuter_model_nowfh_transit
|None|Mix|NoWFH_Mix|commuter_model_nowfh_mix
|Medium|Micro Heavy|MidWFH_Micro|commuter_model_midwfh_micro
|Medium|Transit Heavy|MidWFH_Transit|commuter_model_midwfh_transit
|Medium|Mix|MidWFH_Mix|commuter_model_midwfh_mix
|High|Micro Heavy|HighWFH_Micro|commuter_model_highwfh_micro
|High|Transit Heavy|HighWFH_Transit|commuter_model_highwfh_transit
|High|Mix|HighWFH_Mix|commuter_model_highwfh_mix


In [4]:
commuter_model_nowfh_micro = CommuterModel(ipums_df=ipums_df_19.copy())
commuter_model_nowfh_transit = CommuterModel(ipums_df=ipums_df_19.copy()) 
commuter_model_nowfh_mix = CommuterModel(ipums_df=ipums_df_19.copy())

commuter_model_midwfh_micro = CommuterModel(ipums_df=ipums_df_19.copy())
commuter_model_midwfh_transit = CommuterModel(ipums_df=ipums_df_19.copy())
commuter_model_midwfh_mix = CommuterModel(ipums_df=ipums_df_19.copy())

commuter_model_highwfh_micro = CommuterModel(ipums_df=ipums_df_19.copy())
commuter_model_highwfh_transit = CommuterModel(ipums_df=ipums_df_19.copy())
commuter_model_highwfh_mix = CommuterModel(ipums_df=ipums_df_19.copy())

## No WFH

In [5]:
### ---------------------------------------------------------------------- ### 
## Transit Heavy ## ******** ## ******** ## ******** ## ******** ## ******** 
### ---------------------------------------------------------------------- ### 
commuter_model_nowfh_transit.ipums_df['FLAG_AUTO'] = 0
commuter_model_nowfh_transit.ipums_df['FLAG_ESCOOTER'] = 0
commuter_model_nowfh_transit.ipums_df['FLAG_WALK'] = 0
commuter_model_nowfh_transit.ipums_df['FLAG_EBIKE']= 0
commuter_model_nowfh_transit.ipums_df['FLAG_MOTORCYCLE']= 0
commuter_model_nowfh_transit.ipums_df['FLAG_TAXICAB']= 0
#### Transit
### Bus
commuter_model_nowfh_transit.ipums_df['FLAG_EBUSES'] = commuter_model_nowfh_transit.bus_flag_binary(
                                        home_region=commuter_model_nowfh_transit.puma_home_Bus
                                        ,work_region=commuter_model_nowfh_transit.puma_work_Bus
                                        ,schedule=commuter_model_nowfh_transit.time_Bus
                                        ,affordability=100
                                        ,fixgaps=False
                                         )

## Subway
commuter_model_nowfh_transit.ipums_df['FLAG_SUBWAY'] = commuter_model_nowfh_transit.subway_flag_binary(
                                            home_region=commuter_model_nowfh_transit.puma_home_Subway
                                            ,work_region=commuter_model_nowfh_transit.puma_work_Subway
                                            ,schedule=commuter_model_nowfh_transit.time_Subway
                                            ,affordability=100
                                            ,fixgaps=False
                                            )


## Commuter Rail
commuter_model_nowfh_transit.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_nowfh_transit.commuterRail_flag_binary(
                                             home_region=commuter_model_nowfh_transit.puma_home_CommuterRail
                                            ,work_region=commuter_model_nowfh_transit.puma_work_CommuterRail
                                            ,schedule=commuter_model_nowfh_transit.time_CommuterRail
                                            ,affordability=100
                                            ,fixgaps=False
                                            )

## Ferry
commuter_model_nowfh_transit.ipums_df['FLAG_FERRY'] = commuter_model_nowfh_transit.ferry_flag_binary(
                                            home_region=commuter_model_nowfh_transit.puma_home_Ferry
                                           ,work_region=commuter_model_nowfh_transit.puma_work_Ferry
                                           ,schedule=commuter_model_nowfh_transit.time_Ferry
                                           ,affordability=100
                                           ,fixgaps=False
                                          )
## Zero Work From Home
commuter_model_nowfh_transit.ipums_df["FLAG_WFH"] = 0

### Assign transit mode hierarchically
commuter_model_nowfh_transit.ipums_df['First_Assignment'] = commuter_model_nowfh_transit\
            .AssignTransModesWaterfall(["FLAG_SUBWAY","FLAG_EBUSES","FLAG_COMMUTERRAIL","FLAG_FERRY"])

commuter_model_nowfh_transit.ipums_df['TransMode'] = commuter_model_nowfh_transit.RandAssignChosenOption(
                                                                commuter_model_nowfh_transit.ipums_df['First_Assignment'], 
                                                                commuter_model_nowfh_transit.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

### ---------------------------------------------------------------------- ### 
## Micro Heavy ## ******** ## ******** ## ******** ## ******** ## ******** 
### ---------------------------------------------------------------------- ### 
commuter_model_nowfh_micro.ipums_df['FLAG_AUTO'] = 0
## Motorcycle
commuter_model_nowfh_micro.ipums_df['FLAG_MOTORCYCLE'] = 0
## Taxicab
commuter_model_nowfh_micro.ipums_df['FLAG_TAXICAB'] = 0
## Bus
commuter_model_nowfh_micro.ipums_df['FLAG_EBUSES'] = 0
## Subway
commuter_model_nowfh_micro.ipums_df['FLAG_SUBWAY'] = 0
## Commuter Rail
commuter_model_nowfh_micro.ipums_df['FLAG_COMMUTERRAIL'] = 0
## Ferry
commuter_model_nowfh_micro.ipums_df['FLAG_FERRY'] = 0


## E-Scooter - bump up distance
commuter_model_nowfh_micro.ipums_df['FLAG_ESCOOTER'] = commuter_model_nowfh_micro.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=15
                                            ,scooter_friendly_origins=commuter_model_nowfh_micro.bike_friendly_origins
                                            ,male_pct=100
                                            ,female_pct=100
                                            )


## Walking
commuter_model_nowfh_micro.ipums_df['FLAG_WALK'] = commuter_model_nowfh_micro.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=100
                                            ,female_pct=100
                                            )

## E-Bike
commuter_model_nowfh_micro.ipums_df['FLAG_EBIKE']=commuter_model_nowfh_micro.ebike_flag_binary(
                                        max_age=100
                                        ,max_distance = 24
                                        ,male_pct = 100
                                        ,female_pct = 100
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_nowfh_micro.bike_friendly_origins
                                        )


## Work From Home
commuter_model_nowfh_micro.ipums_df["FLAG_WFH"] = 0
### Assign transit mode hierarchically
commuter_model_nowfh_micro.ipums_df['First_Assignment'] = commuter_model_nowfh_micro.AssignTransModesWaterfall(
                                                    ["FLAG_WALK","FLAG_ESCOOTER","FLAG_EBIKE","FLAG_WFH"]
                                                                                )

commuter_model_nowfh_micro.ipums_df['TransMode'] = commuter_model_nowfh_micro.RandAssignChosenOption(
                                                                commuter_model_nowfh_micro.ipums_df['First_Assignment'], 
                                                                commuter_model_nowfh_micro.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

### ---------------------------------------------------------------------- ### 
## Mixed ## ******** ## ******** ## ******** ## ******** ## ******** 
### ---------------------------------------------------------------------- ### 

commuter_model_nowfh_mix.ipums_df['FLAG_AUTO'] = commuter_model_nowfh_mix.auto_flag_binary(
                                        max_age=70
                                       ,min_distance = 2
                                       ,min_income = 32626
                                       ,male_pct = 15
                                       ,female_pct = 15
                                       ,age_dist = None
                                       )
commuter_model_nowfh_mix.ipums_df['FLAG_MOTORCYCLE'] = commuter_model_nowfh_mix.motorcycle_flag_binary(
                                        max_age=35
                                       ,max_distance = 300
                                       ,min_income = 50000
                                       ,male_pct = 1
                                       ,female_pct = 0
                                       ,age_dist = None
                                        )
commuter_model_nowfh_mix.ipums_df['FLAG_TAXICAB'] = commuter_model_nowfh_mix.taxicab_flag_binary(
                                        max_distance = 15
                                       ,min_income = 32626
                                       ,male_pct = 5
                                       ,female_pct = 2
                                       ,age_dist = None
                                        )
commuter_model_nowfh_mix.ipums_df['FLAG_EBUSES'] = commuter_model_nowfh_mix.bus_flag_binary(
                                        home_region=commuter_model_nowfh_mix.puma_home_Bus
                                        ,work_region=commuter_model_nowfh_mix.puma_work_Bus
                                        ,schedule=commuter_model_nowfh_mix.time_Bus
                                        ,affordability=50
                                        ,fixgaps=False
                                         )


## Subway
commuter_model_nowfh_mix.ipums_df['FLAG_SUBWAY'] = commuter_model_nowfh_mix.subway_flag_binary(
                                            home_region=commuter_model_nowfh_mix.puma_home_Subway
                                            ,work_region=commuter_model_nowfh_mix.puma_work_Subway
                                            ,schedule=commuter_model_nowfh_mix.time_Subway
                                            ,affordability=100
                                            ,fixgaps=False
                                            )



## Commuter Rail
commuter_model_nowfh_mix.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_nowfh_mix.commuterRail_flag_binary(
                                             home_region=commuter_model_nowfh_mix.puma_home_CommuterRail
                                            ,work_region=commuter_model_nowfh_mix.puma_work_CommuterRail
                                            ,schedule=commuter_model_nowfh_mix.time_CommuterRail
                                            ,affordability=100
                                            ,fixgaps=False
                                            )

## Ferry
commuter_model_nowfh_mix.ipums_df['FLAG_FERRY'] = commuter_model_nowfh_mix.ferry_flag_binary(
                                            home_region=commuter_model_nowfh_mix.puma_home_Ferry
                                           ,work_region=commuter_model_nowfh_mix.puma_work_Ferry
                                           ,schedule=commuter_model_nowfh_mix.time_Ferry
                                           ,affordability=100
                                           ,fixgaps=False
                                          )



## E-Scooter
commuter_model_nowfh_mix.ipums_df['FLAG_ESCOOTER'] = commuter_model_nowfh_mix.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=4
                                            ,scooter_friendly_origins=commuter_model_nowfh_mix.bike_friendly_origins
                                            ,male_pct=10
                                            ,female_pct=10
                                            )


## Walking
commuter_model_nowfh_mix.ipums_df['FLAG_WALK'] = commuter_model_nowfh_mix.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=100
                                            ,female_pct=100
                                            )

## E-Bike
commuter_model_nowfh_mix.ipums_df['FLAG_EBIKE']=commuter_model_nowfh_mix.ebike_flag_binary(
                                        max_age=70
                                        ,max_distance = 35
                                        ,male_pct = 50
                                        ,female_pct = 40
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_nowfh_mix.bike_friendly_origins
                                        )


## Work From Home
commuter_model_nowfh_mix.ipums_df["FLAG_WFH"] = 0

## Keep random assignment for mixed transit option
commuter_model_nowfh_mix.ipums_df['First_Assignment'] = commuter_model_nowfh_mix.RandAssignNoOption(
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_WALK'], 
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_nowfh_mix.ipums_df['FLAG_WFH'])
commuter_model_nowfh_mix.ipums_df['TransMode'] = commuter_model_nowfh_mix.RandAssignChosenOption(
                                                                commuter_model_nowfh_mix.ipums_df['First_Assignment'], 
                                                                commuter_model_nowfh_mix.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")


In [6]:
flag_cols = [x for x in commuter_model_nowfh_micro.ipums_df if "FLAG" in x]

In [7]:
commuter_model_nowfh_micro
commuter_model_nowfh_micro.ipums_df[flag_cols].sum()

FLAG_AUTO                0
FLAG_MOTORCYCLE          0
FLAG_TAXICAB             0
FLAG_EBUSES              0
FLAG_SUBWAY              0
FLAG_COMMUTERRAIL        0
FLAG_FERRY               0
FLAG_ESCOOTER         8649
FLAG_WALK             1082
FLAG_EBIKE           14406
FLAG_WFH                 0
dtype: int64

In [8]:
commuter_model_nowfh_transit
commuter_model_nowfh_transit.ipums_df[flag_cols].sum()

FLAG_AUTO                0
FLAG_MOTORCYCLE          0
FLAG_TAXICAB             0
FLAG_EBUSES          13958
FLAG_SUBWAY          17010
FLAG_COMMUTERRAIL     7484
FLAG_FERRY            2516
FLAG_ESCOOTER            0
FLAG_WALK                0
FLAG_EBIKE               0
FLAG_WFH                 0
dtype: int64

In [9]:
commuter_model_nowfh_mix
commuter_model_nowfh_mix.ipums_df[flag_cols].sum()

FLAG_AUTO             1685
FLAG_MOTORCYCLE         37
FLAG_TAXICAB           294
FLAG_EBUSES          13866
FLAG_SUBWAY          17010
FLAG_COMMUTERRAIL     7484
FLAG_FERRY            2516
FLAG_ESCOOTER           95
FLAG_WALK             1082
FLAG_EBIKE            7060
FLAG_WFH                 0
dtype: int64

## Medium WFH
commuter_model_midwfh_micro  
commuter_model_midwfh_transit  
commuter_model_midwfh_mix  

In [10]:
### ---------------------------------------------------------------------- ### 
## Transit Heavy ## ******** ## ******** ## ******** ## ******** ## ******** 
### ---------------------------------------------------------------------- ### 
commuter_model_midwfh_transit.ipums_df['FLAG_AUTO'] = 0
commuter_model_midwfh_transit.ipums_df['FLAG_ESCOOTER'] = 0
commuter_model_midwfh_transit.ipums_df['FLAG_WALK'] = 0
commuter_model_midwfh_transit.ipums_df['FLAG_EBIKE']= 0
commuter_model_midwfh_transit.ipums_df['FLAG_MOTORCYCLE']= 0
commuter_model_midwfh_transit.ipums_df['FLAG_TAXICAB']= 0
#### Transit
### Bus
commuter_model_midwfh_transit.ipums_df['FLAG_EBUSES'] = commuter_model_midwfh_transit.bus_flag_binary(
                                        home_region=commuter_model_midwfh_transit.puma_home_Bus
                                        ,work_region=commuter_model_midwfh_transit.puma_work_Bus
                                        ,schedule=commuter_model_midwfh_transit.time_Bus
                                        ,affordability=100
                                        ,fixgaps=False
                                         )

## Subway
commuter_model_midwfh_transit.ipums_df['FLAG_SUBWAY'] = commuter_model_midwfh_transit.subway_flag_binary(
                                            home_region=commuter_model_midwfh_transit.puma_home_Subway
                                            ,work_region=commuter_model_midwfh_transit.puma_work_Subway
                                            ,schedule=commuter_model_midwfh_transit.time_Subway
                                            ,affordability=100
                                            ,fixgaps=False
                                            )


## Commuter Rail
commuter_model_midwfh_transit.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_midwfh_transit.commuterRail_flag_binary(
                                             home_region=commuter_model_midwfh_transit.puma_home_CommuterRail
                                            ,work_region=commuter_model_midwfh_transit.puma_work_CommuterRail
                                            ,schedule=commuter_model_midwfh_transit.time_CommuterRail
                                            ,affordability=100
                                            ,fixgaps=False
                                            )

## Ferry
commuter_model_midwfh_transit.ipums_df['FLAG_FERRY'] = commuter_model_midwfh_transit.ferry_flag_binary(
                                            home_region=commuter_model_midwfh_transit.puma_home_Ferry
                                           ,work_region=commuter_model_midwfh_transit.puma_work_Ferry
                                           ,schedule=commuter_model_midwfh_transit.time_Ferry
                                           ,affordability=100
                                           ,fixgaps=False
                                          )
## Medium Work From Home
commuter_model_midwfh_transit.ipums_df["FLAG_WFH"] = commuter_model_midwfh_transit\
                                                    .wfh_flag_binary(wfh_dampener = 0.5)

### Assign transit mode hierarchically
commuter_model_midwfh_transit.ipums_df['First_Assignment'] = commuter_model_midwfh_transit\
            .AssignTransModesWaterfall(["FLAG_SUBWAY","FLAG_EBUSES","FLAG_WFH","FLAG_COMMUTERRAIL","FLAG_FERRY"])

commuter_model_midwfh_transit.ipums_df['TransMode'] = commuter_model_midwfh_transit.RandAssignChosenOption(
                                                                commuter_model_midwfh_transit.ipums_df['First_Assignment'], 
                                                                commuter_model_midwfh_transit.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

### ---------------------------------------------------------------------- ### 
## Micro Heavy ## ******** ## ******** ## ******** ## ******** ## ******** 
### ---------------------------------------------------------------------- ### 
commuter_model_midwfh_micro.ipums_df['FLAG_AUTO'] = 0
## Motorcycle
commuter_model_midwfh_micro.ipums_df['FLAG_MOTORCYCLE'] = 0
## Taxicab
commuter_model_midwfh_micro.ipums_df['FLAG_TAXICAB'] = 0
## Bus
commuter_model_midwfh_micro.ipums_df['FLAG_EBUSES'] = 0
## Subway
commuter_model_midwfh_micro.ipums_df['FLAG_SUBWAY'] = 0
## Commuter Rail
commuter_model_midwfh_micro.ipums_df['FLAG_COMMUTERRAIL'] = 0
## Ferry
commuter_model_midwfh_micro.ipums_df['FLAG_FERRY'] = 0


## E-Scooter
commuter_model_midwfh_micro.ipums_df['FLAG_ESCOOTER'] = commuter_model_midwfh_micro.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=15
                                            ,scooter_friendly_origins=commuter_model_midwfh_micro.bike_friendly_origins
                                            ,male_pct=100
                                            ,female_pct=100
                                            )


## Walking
commuter_model_midwfh_micro.ipums_df['FLAG_WALK'] = commuter_model_midwfh_micro.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=100
                                            ,female_pct=100
                                            )

## E-Bike
commuter_model_midwfh_micro.ipums_df['FLAG_EBIKE']=commuter_model_midwfh_micro.ebike_flag_binary(
                                        max_age=100
                                        ,max_distance = 24
                                        ,male_pct = 100
                                        ,female_pct = 100
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_midwfh_micro.bike_friendly_origins
                                        )


## Medium Work From Home
commuter_model_midwfh_micro.ipums_df["FLAG_WFH"] = commuter_model_midwfh_transit\
                                                    .wfh_flag_binary(wfh_dampener = 0.5)

### Assign transit mode hierarchically
commuter_model_midwfh_micro.ipums_df['First_Assignment'] = commuter_model_midwfh_micro.AssignTransModesWaterfall(
                                                    ["FLAG_WALK","FLAG_ESCOOTER","FLAG_EBIKE","FLAG_WFH"]
                                                                                )

commuter_model_midwfh_micro.ipums_df['TransMode'] = commuter_model_midwfh_micro.RandAssignChosenOption(
                                                                commuter_model_midwfh_micro.ipums_df['First_Assignment'], 
                                                                commuter_model_midwfh_micro.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

### ---------------------------------------------------------------------- ### 
## Mixed ## ******** ## ******** ## ******** ## ******** ## ******** 
### ---------------------------------------------------------------------- ### 

commuter_model_midwfh_mix.ipums_df['FLAG_AUTO'] = commuter_model_midwfh_mix.auto_flag_binary(
                                        max_age=70
                                       ,min_distance = 2
                                       ,min_income = 32626
                                       ,male_pct = 15
                                       ,female_pct = 15
                                       ,age_dist = None
                                       )
commuter_model_midwfh_mix.ipums_df['FLAG_MOTORCYCLE'] = commuter_model_midwfh_mix.motorcycle_flag_binary(
                                        max_age=35
                                       ,max_distance = 300
                                       ,min_income = 50000
                                       ,male_pct = 1
                                       ,female_pct = 0
                                       ,age_dist = None
                                        )
commuter_model_midwfh_mix.ipums_df['FLAG_TAXICAB'] = commuter_model_midwfh_mix.taxicab_flag_binary(
                                        max_distance = 15
                                       ,min_income = 32626
                                       ,male_pct = 5
                                       ,female_pct = 2
                                       ,age_dist = None
                                        )
commuter_model_midwfh_mix.ipums_df['FLAG_EBUSES'] = commuter_model_midwfh_mix.bus_flag_binary(
                                        home_region=commuter_model_midwfh_mix.puma_home_Bus
                                        ,work_region=commuter_model_midwfh_mix.puma_work_Bus
                                        ,schedule=commuter_model_midwfh_mix.time_Bus
                                        ,affordability=50
                                        ,fixgaps=False
                                         )


## Subway
commuter_model_midwfh_mix.ipums_df['FLAG_SUBWAY'] = commuter_model_midwfh_mix.subway_flag_binary(
                                            home_region=commuter_model_midwfh_mix.puma_home_Subway
                                            ,work_region=commuter_model_midwfh_mix.puma_work_Subway
                                            ,schedule=commuter_model_midwfh_mix.time_Subway
                                            ,affordability=100
                                            ,fixgaps=False
                                            )



## Commuter Rail
commuter_model_midwfh_mix.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_midwfh_mix.commuterRail_flag_binary(
                                             home_region=commuter_model_midwfh_mix.puma_home_CommuterRail
                                            ,work_region=commuter_model_midwfh_mix.puma_work_CommuterRail
                                            ,schedule=commuter_model_midwfh_mix.time_CommuterRail
                                            ,affordability=100
                                            ,fixgaps=False
                                            )

## Ferry
commuter_model_midwfh_mix.ipums_df['FLAG_FERRY'] = commuter_model_midwfh_mix.ferry_flag_binary(
                                            home_region=commuter_model_midwfh_mix.puma_home_Ferry
                                           ,work_region=commuter_model_midwfh_mix.puma_work_Ferry
                                           ,schedule=commuter_model_midwfh_mix.time_Ferry
                                           ,affordability=100
                                           ,fixgaps=False
                                          )



## E-Scooter
commuter_model_midwfh_mix.ipums_df['FLAG_ESCOOTER'] = commuter_model_midwfh_mix.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=4
                                            ,scooter_friendly_origins=commuter_model_midwfh_mix.bike_friendly_origins
                                            ,male_pct=10
                                            ,female_pct=10
                                            )


## Walking
commuter_model_midwfh_mix.ipums_df['FLAG_WALK'] = commuter_model_midwfh_mix.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=100
                                            ,female_pct=100
                                            )

## E-Bike
commuter_model_midwfh_mix.ipums_df['FLAG_EBIKE']=commuter_model_midwfh_mix.ebike_flag_binary(
                                        max_age=70
                                        ,max_distance = 35
                                        ,male_pct = 50
                                        ,female_pct = 40
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_midwfh_mix.bike_friendly_origins
                                        )


## Medium Work From Home
commuter_model_midwfh_mix.ipums_df["FLAG_WFH"] = commuter_model_midwfh_mix\
                                                    .wfh_flag_binary(wfh_dampener = 0.5)

## Keep random assignment for mixed transit option
commuter_model_midwfh_mix.ipums_df['First_Assignment'] = commuter_model_midwfh_mix.RandAssignNoOption(
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_WALK'], 
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_midwfh_mix.ipums_df['FLAG_WFH'])
commuter_model_midwfh_mix.ipums_df['TransMode'] = commuter_model_midwfh_mix.RandAssignChosenOption(
                                                                commuter_model_midwfh_mix.ipums_df['First_Assignment'], 
                                                                commuter_model_midwfh_mix.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")


In [11]:
commuter_model_midwfh_micro.ipums_df[flag_cols].sum() 

FLAG_AUTO                0
FLAG_MOTORCYCLE          0
FLAG_TAXICAB             0
FLAG_EBUSES              0
FLAG_SUBWAY              0
FLAG_COMMUTERRAIL        0
FLAG_FERRY               0
FLAG_ESCOOTER         8649
FLAG_WALK             1082
FLAG_EBIKE           14406
FLAG_WFH              3424
dtype: int64

In [12]:
commuter_model_midwfh_transit.ipums_df[flag_cols].sum()  

FLAG_AUTO                0
FLAG_MOTORCYCLE          0
FLAG_TAXICAB             0
FLAG_EBUSES          13958
FLAG_SUBWAY          17010
FLAG_COMMUTERRAIL     7484
FLAG_FERRY            2516
FLAG_ESCOOTER            0
FLAG_WALK                0
FLAG_EBIKE               0
FLAG_WFH              3300
dtype: int64

In [13]:
commuter_model_midwfh_mix.ipums_df[flag_cols].sum()  

FLAG_AUTO             1672
FLAG_MOTORCYCLE         32
FLAG_TAXICAB           265
FLAG_EBUSES          13866
FLAG_SUBWAY          17010
FLAG_COMMUTERRAIL     7484
FLAG_FERRY            2516
FLAG_ESCOOTER           92
FLAG_WALK             1082
FLAG_EBIKE            7233
FLAG_WFH              3257
dtype: int64

## High WFH
commuter_model_highwfh_micro  
commuter_model_highwfh_transit  
commuter_model_highwfh_mix  

In [14]:
### ---------------------------------------------------------------------- ### 
## Transit Heavy ## ******** ## ******** ## ******** ## ******** ## ******** 
### ---------------------------------------------------------------------- ### 
commuter_model_highwfh_transit.ipums_df['FLAG_AUTO'] = 0
commuter_model_highwfh_transit.ipums_df['FLAG_ESCOOTER'] = 0
commuter_model_highwfh_transit.ipums_df['FLAG_WALK'] = 0
commuter_model_highwfh_transit.ipums_df['FLAG_EBIKE']= 0
commuter_model_highwfh_transit.ipums_df['FLAG_MOTORCYCLE']= 0
commuter_model_highwfh_transit.ipums_df['FLAG_TAXICAB']= 0
#### Transit
### Bus
commuter_model_highwfh_transit.ipums_df['FLAG_EBUSES'] = commuter_model_highwfh_transit.bus_flag_binary(
                                        home_region=commuter_model_highwfh_transit.puma_home_Bus
                                        ,work_region=commuter_model_highwfh_transit.puma_work_Bus
                                        ,schedule=commuter_model_highwfh_transit.time_Bus
                                        ,affordability=100
                                        ,fixgaps=False
                                         )

## Subway
commuter_model_highwfh_transit.ipums_df['FLAG_SUBWAY'] = commuter_model_highwfh_transit.subway_flag_binary(
                                            home_region=commuter_model_highwfh_transit.puma_home_Subway
                                            ,work_region=commuter_model_highwfh_transit.puma_work_Subway
                                            ,schedule=commuter_model_highwfh_transit.time_Subway
                                            ,affordability=100
                                            ,fixgaps=False
                                            )


## Commuter Rail
commuter_model_highwfh_transit.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_highwfh_transit.commuterRail_flag_binary(
                                             home_region=commuter_model_highwfh_transit.puma_home_CommuterRail
                                            ,work_region=commuter_model_highwfh_transit.puma_work_CommuterRail
                                            ,schedule=commuter_model_highwfh_transit.time_CommuterRail
                                            ,affordability=100
                                            ,fixgaps=False
                                            )

## Ferry
commuter_model_highwfh_transit.ipums_df['FLAG_FERRY'] = commuter_model_highwfh_transit.ferry_flag_binary(
                                            home_region=commuter_model_highwfh_transit.puma_home_Ferry
                                           ,work_region=commuter_model_highwfh_transit.puma_work_Ferry
                                           ,schedule=commuter_model_highwfh_transit.time_Ferry
                                           ,affordability=100
                                           ,fixgaps=False
                                          )
## High Work From Home
commuter_model_highwfh_transit.ipums_df["FLAG_WFH"] = commuter_model_highwfh_transit\
                                                    .wfh_flag_binary(wfh_dampener = 1)

### Assign transit mode hierarchically
commuter_model_highwfh_transit.ipums_df['First_Assignment'] = commuter_model_highwfh_transit\
            .AssignTransModesWaterfall(["FLAG_WFH","FLAG_SUBWAY","FLAG_EBUSES","FLAG_COMMUTERRAIL","FLAG_FERRY"])

commuter_model_highwfh_transit.ipums_df['TransMode'] = commuter_model_highwfh_transit.RandAssignChosenOption(
                                                                commuter_model_highwfh_transit.ipums_df['First_Assignment'], 
                                                                commuter_model_highwfh_transit.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

### ---------------------------------------------------------------------- ### 
## Micro Heavy ## ******** ## ******** ## ******** ## ******** ## ******** 
### ---------------------------------------------------------------------- ### 
commuter_model_highwfh_micro.ipums_df['FLAG_AUTO'] = 0
## Motorcycle
commuter_model_highwfh_micro.ipums_df['FLAG_MOTORCYCLE'] = 0
## Taxicab
commuter_model_highwfh_micro.ipums_df['FLAG_TAXICAB'] = 0
## Bus
commuter_model_highwfh_micro.ipums_df['FLAG_EBUSES'] = 0
## Subway
commuter_model_highwfh_micro.ipums_df['FLAG_SUBWAY'] = 0
## Commuter Rail
commuter_model_highwfh_micro.ipums_df['FLAG_COMMUTERRAIL'] = 0
## Ferry
commuter_model_highwfh_micro.ipums_df['FLAG_FERRY'] = 0


## E-Scooter
commuter_model_highwfh_micro.ipums_df['FLAG_ESCOOTER'] = commuter_model_highwfh_micro.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=15
                                            ,scooter_friendly_origins=commuter_model_highwfh_micro.bike_friendly_origins
                                            ,male_pct=100
                                            ,female_pct=100
                                            )


## Walking
commuter_model_highwfh_micro.ipums_df['FLAG_WALK'] = commuter_model_highwfh_micro.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=100
                                            ,female_pct=100
                                            )

## E-Bike
commuter_model_highwfh_micro.ipums_df['FLAG_EBIKE']=commuter_model_highwfh_micro.ebike_flag_binary(
                                        max_age=100
                                        ,max_distance = 24
                                        ,male_pct = 100
                                        ,female_pct = 100
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_highwfh_micro.bike_friendly_origins
                                        )


## High Work From Home
commuter_model_highwfh_micro.ipums_df["FLAG_WFH"] = commuter_model_highwfh_micro\
                                                    .wfh_flag_binary(wfh_dampener = 1)

### Assign transit mode hierarchically
commuter_model_highwfh_micro.ipums_df['First_Assignment'] = commuter_model_highwfh_micro.AssignTransModesWaterfall(
                                                    ["FLAG_WFH","FLAG_WALK","FLAG_ESCOOTER","FLAG_EBIKE","FLAG_WFH"]
                                                                                )

commuter_model_highwfh_micro.ipums_df['TransMode'] = commuter_model_highwfh_micro.RandAssignChosenOption(
                                                                commuter_model_highwfh_micro.ipums_df['First_Assignment'], 
                                                                commuter_model_highwfh_micro.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

### ---------------------------------------------------------------------- ### 
## Mixed ## ******** ## ******** ## ******** ## ******** ## ******** 
### ---------------------------------------------------------------------- ### 

commuter_model_highwfh_mix.ipums_df['FLAG_AUTO'] = commuter_model_highwfh_mix.auto_flag_binary(
                                        max_age=70
                                       ,min_distance = 2
                                       ,min_income = 32626
                                       ,male_pct = 15
                                       ,female_pct = 15
                                       ,age_dist = None
                                       )
commuter_model_highwfh_mix.ipums_df['FLAG_MOTORCYCLE'] = commuter_model_highwfh_mix.motorcycle_flag_binary(
                                        max_age=35
                                       ,max_distance = 300
                                       ,min_income = 50000
                                       ,male_pct = 1
                                       ,female_pct = 0
                                       ,age_dist = None
                                        )
commuter_model_highwfh_mix.ipums_df['FLAG_TAXICAB'] = commuter_model_highwfh_mix.taxicab_flag_binary(
                                        max_distance = 15
                                       ,min_income = 32626
                                       ,male_pct = 5
                                       ,female_pct = 2
                                       ,age_dist = None
                                        )
commuter_model_highwfh_mix.ipums_df['FLAG_EBUSES'] = commuter_model_highwfh_mix.bus_flag_binary(
                                        home_region=commuter_model_highwfh_mix.puma_home_Bus
                                        ,work_region=commuter_model_highwfh_mix.puma_work_Bus
                                        ,schedule=commuter_model_highwfh_mix.time_Bus
                                        ,affordability=50
                                        ,fixgaps=False
                                         )


## Subway
commuter_model_highwfh_mix.ipums_df['FLAG_SUBWAY'] = commuter_model_highwfh_mix.subway_flag_binary(
                                            home_region=commuter_model_highwfh_mix.puma_home_Subway
                                            ,work_region=commuter_model_highwfh_mix.puma_work_Subway
                                            ,schedule=commuter_model_highwfh_mix.time_Subway
                                            ,affordability=100
                                            ,fixgaps=False
                                            )



## Commuter Rail
commuter_model_highwfh_mix.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_highwfh_mix.commuterRail_flag_binary(
                                             home_region=commuter_model_highwfh_mix.puma_home_CommuterRail
                                            ,work_region=commuter_model_highwfh_mix.puma_work_CommuterRail
                                            ,schedule=commuter_model_highwfh_mix.time_CommuterRail
                                            ,affordability=100
                                            ,fixgaps=False
                                            )

## Ferry
commuter_model_highwfh_mix.ipums_df['FLAG_FERRY'] = commuter_model_highwfh_mix.ferry_flag_binary(
                                            home_region=commuter_model_highwfh_mix.puma_home_Ferry
                                           ,work_region=commuter_model_highwfh_mix.puma_work_Ferry
                                           ,schedule=commuter_model_highwfh_mix.time_Ferry
                                           ,affordability=100
                                           ,fixgaps=False
                                          )



## E-Scooter
commuter_model_highwfh_mix.ipums_df['FLAG_ESCOOTER'] = commuter_model_highwfh_mix.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=4
                                            ,scooter_friendly_origins=commuter_model_highwfh_mix.bike_friendly_origins
                                            ,male_pct=10
                                            ,female_pct=10
                                            )


## Walking
commuter_model_highwfh_mix.ipums_df['FLAG_WALK'] = commuter_model_highwfh_mix.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=100
                                            ,female_pct=100
                                            )

## E-Bike
commuter_model_highwfh_mix.ipums_df['FLAG_EBIKE']=commuter_model_highwfh_mix.ebike_flag_binary(
                                        max_age=70
                                        ,max_distance = 35
                                        ,male_pct = 50
                                        ,female_pct = 40
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_highwfh_mix.bike_friendly_origins
                                        )


## High Work From Home
commuter_model_highwfh_mix.ipums_df["FLAG_WFH"] = commuter_model_highwfh_mix\
                                                    .wfh_flag_binary(wfh_dampener = 1)

## Keep random assignment for mixed transit option
commuter_model_highwfh_mix.ipums_df['First_Assignment'] = commuter_model_highwfh_mix.RandAssignNoOption(
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_WALK'], 
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_highwfh_mix.ipums_df['FLAG_WFH'])
commuter_model_highwfh_mix.ipums_df['TransMode'] = commuter_model_highwfh_mix.RandAssignChosenOption(
                                                                commuter_model_highwfh_mix.ipums_df['First_Assignment'], 
                                                                commuter_model_highwfh_mix.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")


In [15]:
commuter_model_highwfh_micro.ipums_df[flag_cols].sum()  

FLAG_AUTO                0
FLAG_MOTORCYCLE          0
FLAG_TAXICAB             0
FLAG_EBUSES              0
FLAG_SUBWAY              0
FLAG_COMMUTERRAIL        0
FLAG_FERRY               0
FLAG_ESCOOTER         8649
FLAG_WALK             1082
FLAG_EBIKE           14406
FLAG_WFH              6493
dtype: int64

In [16]:
commuter_model_highwfh_transit.ipums_df[flag_cols].sum()    

FLAG_AUTO                0
FLAG_MOTORCYCLE          0
FLAG_TAXICAB             0
FLAG_EBUSES          13958
FLAG_SUBWAY          17010
FLAG_COMMUTERRAIL     7484
FLAG_FERRY            2516
FLAG_ESCOOTER            0
FLAG_WALK                0
FLAG_EBIKE               0
FLAG_WFH              6506
dtype: int64

In [17]:
commuter_model_highwfh_mix.ipums_df[flag_cols].sum()    

FLAG_AUTO             1707
FLAG_MOTORCYCLE         34
FLAG_TAXICAB           295
FLAG_EBUSES          13866
FLAG_SUBWAY          17010
FLAG_COMMUTERRAIL     7484
FLAG_FERRY            2516
FLAG_ESCOOTER           80
FLAG_WALK             1082
FLAG_EBIKE            7235
FLAG_WFH              6411
dtype: int64

In [18]:
### Compare
commuter_model_highwfh_transit.name = "commuter_model_highwfh_transit"

commuter_model_nowfh_micro.name = "commuter_model_nowfh_micro"
commuter_model_nowfh_transit.name = "commuter_model_nowfh_transit"
commuter_model_nowfh_mix.name = "commuter_model_nowfh_mix"
commuter_model_midwfh_micro.name = "commuter_model_midwfh_micro"
commuter_model_midwfh_transit.name = "commuter_model_midwfh_transit"
commuter_model_midwfh_mix.name = "commuter_model_midwfh_mix"
commuter_model_highwfh_micro.name = "commuter_model_highwfh_micro"
commuter_model_highwfh_transit.name = "commuter_model_highwfh_transit"
commuter_model_highwfh_mix.name = "commuter_model_highwfh_mix"

In [19]:
master_cts = pd.DataFrame()
col_names = list()
for model in [commuter_model_nowfh_micro
            ,commuter_model_nowfh_transit
            ,commuter_model_nowfh_mix
            ,commuter_model_midwfh_micro
            ,commuter_model_midwfh_transit
            ,commuter_model_midwfh_mix
            ,commuter_model_highwfh_micro
            ,commuter_model_highwfh_transit
            ,commuter_model_highwfh_mix]:
    model_cts = model.ipums_df['First_Assignment'].value_counts(dropna=False)
    master_cts = pd.concat([master_cts,model_cts],axis=1)
    col_names.append(model.name)
master_cts.columns = col_names
master_cts

,commuter_model_nowfh_micro,commuter_model_nowfh_transit,commuter_model_nowfh_mix,commuter_model_midwfh_micro,commuter_model_midwfh_transit,commuter_model_midwfh_mix,commuter_model_highwfh_micro,commuter_model_highwfh_transit,commuter_model_highwfh_mix
No Option,9572.0,1674.0,1412.0,8046.0,1464.0,1253,6661.0,1249.0,1038
Escooter,7740.0,NaN,22.0,7740.0,NaN,26,5911.0,NaN,28
Bicycle,5584.0,NaN,2556.0,5584.0,NaN,2485,4091.0,NaN,2466
Walk,1082.0,NaN,328.0,1082.0,NaN,325,822.0,NaN,309
Subway,NaN,17010.0,8047.0,NaN,17010.0,7744,NaN,12744.0,7452
CommuterRail,NaN,2745.0,4191.0,NaN,2151.0,3869,NaN,1588.0,3602
Bus,NaN,2511.0,5603.0,NaN,2511.0,5360,NaN,1862.0,5079
Ferry,NaN,38.0,975.0,NaN,33.0,884,NaN,29.0,835
AutoOccupants,NaN,NaN,735.0,NaN,NaN,697,NaN,NaN,653
Taxicab,NaN,NaN,92.0,NaN,NaN,72,NaN,NaN,74


## Electric Model

### Iterations 
|WFH Status|Commuter Mode|Save Directory Name|Commuter Model Object| Electric Model Object|
|----|-----|-----|-----|-----|
|None|Micro Heavy|NoWFH_Micro|commuter_model_nowfh_micro|electric_model_nowfh_micro|
|None|Transit Heavy|NoWFH_Transit|commuter_model_nowfh_transit|electric_model_nowfh_transit|
|None|Mix|NoWFH_Mix|commuter_model_nowfh_mix|electric_model_nowfh_mix|
|Medium|Micro Heavy|MidWFH_Micro|commuter_model_midwfh_micro|electric_model_midwfh_micro|
|Medium|Transit Heavy|MidWFH_Transit|commuter_model_midwfh_transit|electric_model_midwfh_transit|
|Medium|Mix|MidWFH_Mix|commuter_model_midwfh_mix|electric_model_midwfh_mix|
|High|Micro Heavy|HighWFH_Micro|commuter_model_highwfh_micro|electric_model_highwfh_micro|
|High|Transit Heavy|HighWFH_Transit|commuter_model_highwfh_transit|electric_model_highwfh_transit|
|High|Mix|HighWFH_Mix|commuter_model_highwfh_mix|electric_model_highwfh_mix|

In [20]:
electric_model_nowfh_micro = ElectricModel(commuter_model_nowfh_micro.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_nowfh_micro')

electric_model_nowfh_transit = ElectricModel(commuter_model_nowfh_transit.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_nowfh_transit')

electric_model_nowfh_mix = ElectricModel(commuter_model_nowfh_mix.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_nowfh_mix')

electric_model_midwfh_micro = ElectricModel(commuter_model_midwfh_micro.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_midwfh_micro')

electric_model_midwfh_transit = ElectricModel(commuter_model_midwfh_transit.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_midwfh_transit')

electric_model_midwfh_mix = ElectricModel(commuter_model_midwfh_mix.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_midwfh_mix')

electric_model_highwfh_micro = ElectricModel(commuter_model_highwfh_micro.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_highwfh_micro')

electric_model_highwfh_transit = ElectricModel(commuter_model_highwfh_transit.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_highwfh_transit')

electric_model_highwfh_mix = ElectricModel(commuter_model_highwfh_mix.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'electric_model_highwfh_mix')

In [21]:
ct= 0 
for e_m in [electric_model_nowfh_micro
    ,electric_model_nowfh_transit
    ,electric_model_nowfh_mix
    ,electric_model_midwfh_micro
    ,electric_model_midwfh_transit
    ,electric_model_midwfh_mix
    ,electric_model_highwfh_micro
    ,electric_model_highwfh_transit
    ,electric_model_highwfh_mix]:
    ct+=1
    
    ## Run methods in order to read in, clean, and aggregate data
    e_m.read_electric_info_in()

    e_m.clean_commuter_df()

    e_m.create_in_out_flow_dfs()

    e_m.aggregate_data()
    print(f"electric model done for em {ct}: {e_m.model_name}")

ev reference table read in. subway floor and commuter rail floor extracted from table
electric model done for em 1: electric_model_nowfh_micro
ev reference table read in. subway floor and commuter rail floor extracted from table
electric model done for em 2: electric_model_nowfh_transit
ev reference table read in. subway floor and commuter rail floor extracted from table
electric model done for em 3: electric_model_nowfh_mix
ev reference table read in. subway floor and commuter rail floor extracted from table
electric model done for em 4: electric_model_midwfh_micro
ev reference table read in. subway floor and commuter rail floor extracted from table
electric model done for em 5: electric_model_midwfh_transit
ev reference table read in. subway floor and commuter rail floor extracted from table
electric model done for em 6: electric_model_midwfh_mix
ev reference table read in. subway floor and commuter rail floor extracted from table
electric model done for em 7: electric_model_highwfh_

In [22]:
save_dirs = ["NoWFH_Micro"
,"NoWFH_Transit"
,"NoWFH_Mix"
,"MidWFH_Micro"
,"MidWFH_Transit"
,"MidWFH_Mix"
,"HighWFH_Micro"
,"HighWFH_Transit"
,"HighWFH_Mix"]

commuter_models = [commuter_model_nowfh_micro
            ,commuter_model_nowfh_transit
            ,commuter_model_nowfh_mix
            ,commuter_model_midwfh_micro
            ,commuter_model_midwfh_transit
            ,commuter_model_midwfh_mix
            ,commuter_model_highwfh_micro
            ,commuter_model_highwfh_transit
            ,commuter_model_highwfh_mix]

electric_models = [electric_model_nowfh_micro
    ,electric_model_nowfh_transit
    ,electric_model_nowfh_mix
    ,electric_model_midwfh_micro
    ,electric_model_midwfh_transit
    ,electric_model_midwfh_mix
    ,electric_model_highwfh_micro
    ,electric_model_highwfh_transit
    ,electric_model_highwfh_mix]



In [27]:
save_dir = "../06_Dashboard_Visualization/electric_model_outputs_WFHTransitCombos"
os.makedirs(save_dir,exist_ok=True)

for save_loc,commuter_model,electric_model in zip(save_dirs,commuter_models,electric_models):
    save_loc_models = f"{save_dir}/{save_loc}"
    print(save_loc_models)
    print(commuter_model.name," | ",electric_model.model_name)
    os.makedirs(save_loc_models,exist_ok=True)
    
    commuter_model.ipums_df.to_pickle(f"{save_loc_models}/commuter_model_ipums_df.pkl")
    
    electric_model.df_aggregate.to_pickle(f"{save_loc_models}/electric_model_df_aggregate.pkl")
    print()

../06_Dashboard_Visualization/electric_model_outputs_WFHTransitCombos/NoWFH_Micro
commuter_model_nowfh_micro  |  electric_model_nowfh_micro

../06_Dashboard_Visualization/electric_model_outputs_WFHTransitCombos/NoWFH_Transit
commuter_model_nowfh_transit  |  electric_model_nowfh_transit

../06_Dashboard_Visualization/electric_model_outputs_WFHTransitCombos/NoWFH_Mix
commuter_model_nowfh_mix  |  electric_model_nowfh_mix

../06_Dashboard_Visualization/electric_model_outputs_WFHTransitCombos/MidWFH_Micro
commuter_model_midwfh_micro  |  electric_model_midwfh_micro

../06_Dashboard_Visualization/electric_model_outputs_WFHTransitCombos/MidWFH_Transit
commuter_model_midwfh_transit  |  electric_model_midwfh_transit

../06_Dashboard_Visualization/electric_model_outputs_WFHTransitCombos/MidWFH_Mix
commuter_model_midwfh_mix  |  electric_model_midwfh_mix

../06_Dashboard_Visualization/electric_model_outputs_WFHTransitCombos/HighWFH_Micro
commuter_model_highwfh_micro  |  electric_model_highwfh_micro